In [1]:
!pip install carla

Looking in indexes: https://pypi.org/simple, https://pypi.idi.ntnu.no


In [42]:
import carla
import random
import time
import numpy as np
import cv2

In [35]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [36]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [40]:
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
#vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[79])

In [44]:
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=3.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

AttributeError: 'NoneType' object has no attribute 'get_transform'

In [22]:
for i in range(30):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))


In [14]:
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [6]:
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)
#time.sleep(0.2)
#spectator.set_transform(camera.get_transform())
#camera.destroy()

In [9]:
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [1]:
camera.stop()

NameError: name 'camera' is not defined

In [16]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [13]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

In [15]:
vehicle.set_autopilot(True)

In [15]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)
while True:
    cv2.imshow('RGB Camera', camera_data['image'])
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [7]:
for bp in bp_lib.filter('sensor'):
    print(bp.id)

sensor.other.lane_invasion
sensor.other.collision
sensor.camera.depth
sensor.lidar.ray_cast
sensor.camera.semantic_segmentation
sensor.other.radar
sensor.camera.instance_segmentation
sensor.camera.rgb
sensor.other.rss
sensor.camera.optical_flow
sensor.lidar.ray_cast_semantic
sensor.other.obstacle
sensor.other.imu
sensor.other.gnss
sensor.camera.dvs
sensor.camera.normals


In [23]:
camera_init_trans = carla.Transform(carla.Location(z=2))

camera_bp = bp_lib.find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle) 

inst_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle) 

depth_camera_bp = bp_lib.find('sensor.camera.depth')
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle) 

dvs_camera_bp = bp_lib.find('sensor.camera.dvs')
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=vehicle) 

opt_camera_bp = bp_lib.find('sensor.camera.optical_flow')
opt_camera = world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to=vehicle) 

In [24]:
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def inst_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def opt_callback(data, data_dict):
    image = data.get_color_coded_flow()
    img =  np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img[:,:,3]=255
    data_dict['opt_image'] = img


def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([('x', np.uint16), ('y', np.uint16),('t', np.int64), ('pol', np.bool_)]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol']*2]=255
    data_dict['dvs_image'][:,:,0:3]=dvs_img



In [25]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4)),
               'depth_image': np.zeros((image_h, image_w, 4)),
               'dvs_image': np.zeros((image_h, image_w, 4)),
               'opt_image': np.zeros((image_h, image_w, 4)),
               'inst_image': np.zeros((image_h, image_w, 4))
               }




cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1)
tiled = np.concatenate((top_row, lower_row), axis=0)

cv2.imshow('All cameras', tiled)
cv2.waitKey(1)

camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))
opt_camera.listen(lambda image: opt_callback(image, sensor_data))


while True:
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
    lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']),axis=1)
    tiled = np.concatenate((top_row, lower_row), axis=0)
    cv2.imshow('All cameras', tiled)
    if cv2.waitKey(1) == ord('q'):
        break


camera.stop()
sem_camera.stop()
inst_camera.stop()
depth_camera.stop()
dvs_camera.stop()
opt_camera.stop()
cv2.destroyAllWindows()


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 4492, resource id: 0, major code: 140 (Unknown), minor code: 20


KeyboardInterrupt: 